In [ ]:
repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys


if repo_root not in sys.path:
    sys.path.append(repo_root)  # go to parent dir

In [ ]:
from torch.utils.data import Dataset
import yaml
import torch
from spf.rf import precompute_steering_vectors
from spf.utils import zarr_open_from_lmdb_store
from spf.dataset.v5_data import v5rx_f64_keys, v5rx_2xf64_keys
import numpy as np
from spf.rf import speed_of_light
from multiprocessing.pool import ThreadPool
import os
import pickle
from multiprocessing import Pool
import time
import tqdm

#         return pickle.load(open(results_fn, "rb"))

In [ ]:
from spf.dataset.spf_dataset import v5spfdataset


ds = v5spfdataset(
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_04_05_22_13_07_nRX2_rx_circle"
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_05_10_05_03_21_nRX2_rx_circle_tag_rand10_90_rand30_100"
    # "/Volumes/SPFData/missions/april5/wallarrayv3_2024_04_10_05_08_55_nRX2_rx_circle"
    "/Volumes/SPFData/missions/april5/wallarrayv3_2024_05_06_19_04_15_nRX2_bounce",
    nthetas=11,
)

In [ ]:
ds[0]

In [ ]:
import glob


# for x in glob.glob('/Volumes/SPFData/missions/april5/wallarrayv3_2024_05*.zarr'):
#     print(x)
#     ds=v5spfdataset(x)
#     ds.get_segmentation()

In [ ]:
# for x in glob.glob('/Volumes/SPFData/missions/april5/wallarrayv3_2024_04*.zarr'):
#     print(x)
#     ds=v5spfdataset(x)
#     ds.get_segmentation()

In [ ]:
_ = ds.get_segmentation()

In [ ]:
_ = ds.get_segmentation()

In [ ]:
ds[0]["signal_matrix"].shape

In [ ]:
np.array([1, 2]).shape

In [ ]:
session_idx = 150 * 2

In [ ]:
from spf.rf import beamformer, beamformer_given_steering_nomean, get_phase_diff
import matplotlib.pyplot as plt

data = ds[session_idx]
# signal_matrix = load_zarr_to_numpy(z.receivers["r0"].signal_matrix[session_idx])
n = 2 * 4 * 50000
offset = 0
v = data["signal_matrix"][:, offset : offset + n].numpy()
pd = get_phase_diff(v)

fig, axs = plt.subplots(2, 1, figsize=(12, 6))

axs[0].scatter(np.arange(n), np.abs(v[0]), alpha=0.1, s=1)
axs[0].scatter(np.arange(n), np.abs(v[1]), alpha=0.1, s=1)
axs[0].set_title("Raw signal")
axs[0].set_xlabel("Sample# (time)")
axs[1].set_xlabel("Sample# (time)")
axs[1].set_title("Phase estimates")
axs[1].scatter(np.arange(n), pd, s=1, alpha=0.1)
from spf.sdrpluto.sdr_controller import circular_mean, circular_stddev, simple_segment

beam_sds = [
    beamformer_given_steering_nomean(
        steering_vectors=ds.steering_vectors[receiver_idx],
        signal_matrix=v,
    )
    for receiver_idx in range(2)
]

window_sds = []
for window in simple_segment(
    v,
    window_size=2500,
    stride=2500,
    trim=20,
    mean_diff_threshold=0.2,  #
    max_stddev_threshold=0.5,  # just eyeballed this
    drop_less_than_size=3000,
    min_abs_signal=40,
):
    if window["type"] == "signal":
        axs[1].plot(
            [window["start_idx"], window["end_idx"]],
            [window["mean"], window["mean"]],
            color="red",
        )
    else:
        axs[1].plot(
            [window["start_idx"], window["end_idx"]],
            [window["mean"], window["mean"]],
            color="orange",
        )
    # print(window["start_idx"], window["end_idx"])
    _beam_sds = beam_sds[0][:, window["start_idx"] : window["end_idx"]].mean(axis=1)
    # _beam_sds = _beam_sds.mean(axis=1)
    # _beam_sds -= _beam_sds.min()
    window_sds.append(_beam_sds)
window_sds = np.array(window_sds)
fig.set_tight_layout(True)

In [ ]:
segmentation_by_receiver = ds.get_segmentation()
mean_phase_results = {}
for receiver, results in segmentation_by_receiver.items():
    mean_phase_results[receiver] = np.array(
        [np.array([x["mean"] for x in result]).mean() for result in results]
    )


first_n = 250 * 2
fig, axs = plt.subplots(1, 1)
axs.scatter(range(first_n), mean_phase_results["r0"][:first_n], s=3, label="Rx0")
axs.scatter(range(first_n), mean_phase_results["r1"][:first_n], s=3, label="Rx1")
axs.legend()
axs.axvline(x=115)
axs.set_title("Mean segmented phase diff")
axs.set_xlabel("Chunk (time)")
axs.set_ylabel("Mean phase diff of seg. chunk")

In [ ]:
ds.get_estimated_thetas()

In [ ]:
from spf.dataset.spf_dataset import pi_norm
from spf.rf import c as speed_of_light


fig, axs = plt.subplots(1, 2, figsize=(12, 4))

estimated_thetas = ds.get_estimated_thetas()
for rx_idx in [0, 1]:

    axs[rx_idx].scatter(
        range(estimated_thetas[f"r{rx_idx}"][0].shape[0]),
        pi_norm(estimated_thetas[f"r{rx_idx}"][0]),
        s=0.4,
    )
    axs[rx_idx].scatter(
        range(estimated_thetas[f"r{rx_idx}"][1].shape[0]),
        pi_norm(estimated_thetas[f"r{rx_idx}"][1]),
        s=0.4,
    )
    axs[rx_idx].scatter(
        range(estimated_thetas[f"r{rx_idx}"][2].shape[0]),
        pi_norm(estimated_thetas[f"r{rx_idx}"][2]),
        s=0.4,
    )
    axs[rx_idx].set_xlabel("Chunk")
    axs[rx_idx].set_ylabel("estimated theta")

In [ ]:
from spf.dataset.spf_dataset import pi_norm


fig, axs = plt.subplots(1, 2, figsize=(12, 4))

first_n = 250 * 2
estimated_thetas = ds.get_estimated_thetas()
for rx_idx in [0, 1]:
    expected_theta = ds.ground_truth_thetas[rx_idx]
    axs[rx_idx].plot(
        expected_theta[:first_n], alpha=1, color="red", label="ground truth"
    )

    n = estimated_thetas[f"r{rx_idx}"][0].shape[0]
    axs[rx_idx].scatter(
        range(first_n),
        pi_norm(estimated_thetas[f"r{rx_idx}"][0])[:first_n],
        s=3,
        label=f"Rx{rx_idx}_peak1",
    )
    axs[rx_idx].scatter(
        range(first_n),
        pi_norm(estimated_thetas[f"r{rx_idx}"][1])[:first_n],
        s=3,
        label=f"Rx{rx_idx}_peak2",
    )
    axs[rx_idx].set_xlabel("Chunk")
    axs[rx_idx].set_ylabel("estimated theta")
    axs[rx_idx].legend()
    axs[rx_idx].set_title(f"Receiver (Rx) {rx_idx}")

In [ ]:
from collections import OrderedDict

import torch
import torch.nn as nn

In [ ]:
from functools import cache
import gc

from spf.dataset.spf_dataset import v5_collate_beamsegnet
from spf.model_training_and_inference.models.beamsegnet import BeamNSegNet

torch_device = torch.device("mps")
nthetas = 11
lr = 0.001


dataloader_params = {
    "batch_size": 4,
    "shuffle": True,
    "num_workers": 1,
    "collate_fn": v5_collate_beamsegnet,
}
train_dataloader = torch.utils.data.DataLoader(ds, **dataloader_params)

import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="projectspf",
    # track hyperparameters and run metadata
    config={
        "learning_rate": lr,
        "architecture": "beamsegnet1",
    },
)


import torch.nn.functional as f


@cache
def mean_guess(shape):
    return f.normalize(torch.ones(shape), p=1, dim=1)


loss_fn = nn.MSELoss()
m = BeamNSegNet(nthetas=nthetas).to(torch_device)
optimizer = torch.optim.AdamW(m.parameters(), lr=0.01)
for epoch in range(10):
    for X, Y in train_dataloader:
        optimizer.zero_grad()
        output = m(X.to(torch_device))
        l = loss_fn(output, Y.to(torch_device))
        l.backward()
        mean_loss = output
        optimizer.step()

        mean_loss = loss_fn(mean_guess(Y.shape), Y)

        wandb.log({"loss": l.item(), "mean_loss": mean_loss.item()})
        # print("LOSS", l.item())


# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

In [ ]:
output

In [ ]:
plt.imshow(output.cpu().detach().numpy())

In [ ]:
plt.imshow(Y.to("cpu"))